In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from matplotlib.pyplot import scatter

In [2]:
topics = ['ukraine', 'covid', 'costofliving', 'brexit']

In [3]:
df = pd.DataFrame()

for topic in topics:

    df_pc = pd.read_csv('PCA/'+topic+'_individual.csv', index_col=0)

    df_pc['topic'] = topics.index(topic)

    df = pd.concat([df,df_pc])    

In [4]:
df['Inherent-Active'] = df['Inherent']*df['Active']
df = df.groupby(['Label_str','week','topic']).mean()

df = df.reset_index()

In [5]:
for index,row in df.iterrows():
    
    topic = topics[row['topic']]

    lbl = row['Label_str']
    week = row['week']
    
    df_te = pd.read_csv('Transfer_entropy/'+topic+'.csv', index_col=0)

    df.at[index,'TE'] = df_te.loc[lbl]['Week'+str(week)]
    
df['Intercept'] = 1

In [8]:
model = sm.MixedLM(df['TE'],df[['Inherent','Active','Inherent-Active','week','Intercept']],groups=df['topic'])
results = model.fit()

/Users/schakraborty/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [9]:
results.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
           Mixed Linear Model Regression Results
===========================================================
Model:              MixedLM  Dependent Variable:  TE       
No. Observations:   1045     Method:              REML     
No. Groups:         4        Scale:               0.0042   
Min. group size:    233      Log-Likelihood:      1337.2437
Max. group size:    297      Converged:           Yes      
Mean group size:    261.2                                  
-----------------------------------------------------------
                Coef.  Std.Err.    z    P>|z| [0.025 0.975]
-----------------------------------------------------------
Inherent         0.010    0.001   7.658 0.000  0.008  0.013
Active           0.008    0.004   2.246 0.025  0.001  0.016
Inherent-Active -0.001    0.001  -1.593 0.111 -0.003  0.000
week            -0.002    0.000 -10.744 0.000 -0.002 -0.001
Intercept        0.088    0.029   3.088 0.002  0.032  0.145
Group Var        0.003    0.041                            
===========================================================

"""